# Proyecto I - Aprendizaje Máquina

## Integrantes:
- **Cristobal Hachim**
- **Ignacio Duarte**
- **Cristopher Arredondo**

## Introducción
Los Sistemas de Grupos Relacionados por el Diagnóstico (GRD) constituyen un método estandarizado para agrupar estancias hospitalarias en categorías clínicas homogéneas, cuyo objetivo es reflejar la complejidad del paciente y el consumo de recursos asociados. Tradicionalmente, la asignación de un GRD se realiza al alta mediante reglas fijas basadas en el diagnóstico principal, las comorbilidades, los procedimientos realizados, la edad y el sexo, lo que limita su capacidad de adaptación a nuevas prácticas clínicas o combinaciones atípicas de diagnósticos. Predecir de forma temprana el GRD —incluso al ingreso o durante las primeras horas de hospitalización— permitiría a los hospitales anticipar necesidades de camas, personal y suministros, optimizar la planificación presupuestaria y detectar posibles errores de codificación antes de la facturación. Sin embargo, esta tarea plantea un desafío de clasificación multiclase de alta dimensionalidad, dado el enorme número de códigos ICD‑9/ICD‑10 y procedimientos, así como las complejas interacciones entre comorbilidades y tratamientos.

---

**Posible Cambio**

---
Por ello, en este proyecto exploraremos desde técnicas clásicas (regresión logística, árboles de decisión) hasta métodos avanzados de aprendizaje automático (Random Forest, XGBoost, redes neuronales recurrentes y basadas en atención), con el fin de diseñar un sistema preciso, escalable y lo suficientemente interpretable para apoyar la toma de decisiones clínicas y administrativas.

---